In [1]:
import numpy as np
import cv2
import json
import os
import random
import itertools


In [2]:
def generate_combinations(lst):
    combinations = []
    n = len(lst)
    for r in range(n, 0, -1):
        for combo in itertools.combinations(lst, r):
            combinations.append(combo)
    return combinations

def create_segmentation_mask(image_shape, removing_info):
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    for obj in removing_info:
        polygon = np.array(obj['segmentation'], dtype=np.int32)
        cv2.fillPoly(mask, [polygon], 255)
    return mask

def remove_person_and_fill_background(image, mask, background_image):
    background_image = cv2.resize(background_image, (image.shape[1], image.shape[0]))
    image[mask == 255] = background_image[mask == 255]
    return image

def apply_color_jitter(image, mask, brightness=0, contrast=0, saturation=0, hue=0):
    # Convert image to float32
    jittered_image = image.astype(np.float32) / 255.0
    
    # Apply brightness change
    if brightness > 0:
        alpha = 1.0 + random.uniform(-brightness, brightness)
        jittered_image = jittered_image * alpha
        jittered_image = np.clip(jittered_image, 0, 1)
    
    # Apply contrast change
    if contrast > 0:
        alpha = 1.0 + random.uniform(-contrast, contrast)
        jittered_image = jittered_image * alpha
        jittered_image = np.clip(jittered_image, 0, 1)

    # Apply saturation change
    if saturation > 0:
        jittered_image_hsv = cv2.cvtColor(jittered_image, cv2.COLOR_BGR2HSV)
        jittered_image_hsv[:, :, 1] *= 1.0 + random.uniform(-saturation, saturation)
        jittered_image_hsv[:, :, 1] = np.clip(jittered_image_hsv[:, :, 1], 0, 1)
        jittered_image = cv2.cvtColor(jittered_image_hsv, cv2.COLOR_HSV2BGR)

    # Apply hue change
    if hue > 0:
        jittered_image_hsv = cv2.cvtColor(jittered_image, cv2.COLOR_BGR2HSV)
        jittered_image_hsv[:, :, 0] += random.uniform(-hue, hue) * 180
        jittered_image_hsv[:, :, 0] = np.mod(jittered_image_hsv[:, :, 0], 180)
        jittered_image = cv2.cvtColor(jittered_image_hsv, cv2.COLOR_HSV2BGR)

    jittered_image = (jittered_image * 255).astype(np.uint8)

    # Apply the changes only to the masked area
    image[mask == 255] = jittered_image[mask == 255]
    return image

def check_folder(path):
    if os.path.exists(path) is False:
        os.mkdir(path)
        print(f"create {path} folder.")

def select_background(image_id):
    # 첫번째 배경: 0 ~ 61,  기준 25 / 29 사용
    # 두번째 배경: 124~179, 기준 132 / 129 사용
    # 세번째 배경: 237~283, 기준 243 / 270 사용
    # 네번째 배경: 332~393, 기준 357 / 376 사용
    # if image_id < 62:
    #     return 0
    # elif image_id < 180:
    #     return 1
    # elif image_id < 284:
    #     return 2
    # else:
    #     return 3
    
    if image_id < 124:
        return 0
    elif image_id < 237:
        return 1
    elif image_id < 331:
        return 2
    else:
        return 3





In [3]:
with open('datasets/train.json') as f:
    data = json.load(f)

# Directory containing images
image_dir = './datasets/yolo_dataset/images/origin/train'
image_files = os.listdir(image_dir)

yolo_validation_data_dir = './datasets/yolo_dataset/images/origin/val'


# target_dir = '/workspace/CV_project/data/cv_data/remove_test'
target_dir = './datasets/yolo_dataset/remove'
# target_dir = '/workspace/CV_project/data/cv_data/distortion_test'
check_folder(target_dir)


validation_files = os.listdir(yolo_validation_data_dir)

check_folder(os.path.join(target_dir,'images'))
check_folder(os.path.join(target_dir,'labels'))

target_image_dir = os.path.join(target_dir,'images/train')
target_label_dir = os.path.join(target_dir,'labels/train')


check_folder(target_image_dir)
check_folder(target_label_dir)

background_images = []
background_images.append(cv2.imread(os.path.join(image_dir,'b1.png')))
background_images.append(cv2.imread(os.path.join(image_dir,'b2.png')))
background_images.append(cv2.imread(os.path.join(image_dir,'b3.png')))
background_images.append(cv2.imread(os.path.join(image_dir,'b4.png')))


create ./datasets/yolo_dataset/remove folder.
create ./datasets/yolo_dataset/remove/images folder.
create ./datasets/yolo_dataset/remove/labels folder.
create ./datasets/yolo_dataset/remove/images/train folder.
create ./datasets/yolo_dataset/remove/labels/train folder.


In [4]:
# Process each image
W = 640
H = 345

# label이 있는 전체 파일 대상
n = 0
origin_count = 0
for image_file in image_files:
    
    if 'b' in image_file:
        continue
    
    if image_file in validation_files:
        print('valid ', image_file)
        continue
    
    # print(f"{image_file} : ", end='')
    image_path = os.path.join(image_dir, image_file)
    image_id = int(os.path.splitext(image_file)[0])  # Assuming image file name is the image_id
    segmentation_info = []

    W, H = data['images'][image_id]['width'], data['images'][image_id]['height']
    save_person_id = []
    
    for ann in data['annotations']:
        if ann['image_id'] == image_id:
            person_id = ann['category_id']
            save_person_id.append(person_id)
            segmentation = ann['segmentation']
            bbox = ann['bbox']
            segmentation_info.append({'category_id': person_id, 'segmentation': segmentation, 'bbox': bbox})
    
    if len(save_person_id)  == 0:
        continue
    
    # save_person_id 중복제거
    save_person_id = list(set(save_person_id))
    
    # 배경선택
    background_id = select_background(image_id)
    
    # 제거할 사람 id 선택
    combinations = generate_combinations(save_person_id)
    s = 0.5
    origin_count+=1
    
    for i in range(len(combinations)):
        if i == 0:
            continue
        n += 1
        
        image = cv2.imread(image_path)
        removing_id = tuple(combinations[i])
        removing_info = [info for info in segmentation_info if info['category_id'] in removing_id]
        remaining_info = [info for info in segmentation_info if info['category_id'] not in removing_id]
        
        mask = create_segmentation_mask(image.shape, removing_info)
        result_image = remove_person_and_fill_background(image, mask, background_images[background_id])
        # result_image = apply_color_jitter(image, mask, brightness=0.8 * s, contrast=0.8 * s, saturation=0.8 * s, hue=0.2 * s)
        number_of_file = image_file.replace('.png','')
        save_name = 're_'+number_of_file+'_'+str(i)+'.png'
        
        output_path = os.path.join(target_image_dir,save_name)
        cv2.imwrite(output_path, result_image)
        
        label_path = os.path.join(target_label_dir,save_name.replace('png','txt'))
        with open(label_path, 'w') as f:
            for info in remaining_info:
                category_id = info['category_id']
                bbox = info['bbox']
                x = (bbox[0]+bbox[2]/2) / W
                y = (bbox[1]+bbox[3]/2) / H
                w = bbox[2] / W
                h = bbox[3] / H
                
                f.write(f'{category_id} {x} {y} {w} {h}\n')
                
                
print("number of train set :", n)
print("number of origin images :", origin_count)

number of train set : 31376
number of origin images : 180


In [5]:
# Process each image
W = 640
H = 345

# label이 있는 전체 파일 대상
n = 0
for image_file in image_files:
    
    if 'b' in image_file:
        continue
    
    if image_file in validation_files:
        print('valid ', image_file)
        continue
    
    # print(f"{image_file} : ", end='')
    image_path = os.path.join(image_dir, image_file)
    image_id = int(os.path.splitext(image_file)[0])  # Assuming image file name is the image_id
    segmentation_info = []

    W, H = data['images'][image_id]['width'], data['images'][image_id]['height']
    save_person_id = []
    
    for ann in data['annotations']:
        if ann['image_id'] == image_id:
            person_id = ann['category_id']
            save_person_id.append(person_id)
            segmentation = ann['segmentation']
            bbox = ann['bbox']
            segmentation_info.append({'category_id': person_id, 'segmentation': segmentation, 'bbox': bbox})
    
    if len(save_person_id)  == 0:
        continue
    
    # save_person_id 중복제거
    save_person_id = list(set(save_person_id))
    
    # 배경선택
    background_id = select_background(image_id)
    
    # 제거할 사람 id 선택
    combinations = generate_combinations(save_person_id)
    s = 0.5
    
    
    for i in range(len(combinations)):
        if i == 0:
            continue
        n += 1
        continue
        image = cv2.imread(image_path)
        removing_id = tuple(combinations[i])
        removing_info = [info for info in segmentation_info if info['category_id'] in removing_id]
        remaining_info = [info for info in segmentation_info if info['category_id'] not in removing_id]
        
        mask = create_segmentation_mask(image.shape, removing_info)
        # result_image = remove_person_and_fill_background(image, mask, background_images[background_id])
        result_image = apply_color_jitter(image, mask, brightness=0.8 * s, contrast=0.8 * s, saturation=0.8 * s, hue=0.2 * s)
        number_of_file = image_file.replace('.png','')
        save_name = 'di_'+number_of_file+'_'+str(i)+'.png'
        
        output_path = os.path.join(target_image_dir,save_name)
        cv2.imwrite(output_path, result_image)
        
        label_path = os.path.join(target_label_dir,save_name.replace('png','txt'))
        with open(label_path, 'w') as f:
            for info in remaining_info:
                category_id = info['category_id']
                bbox = info['bbox']
                x = (bbox[0]+bbox[2]/2) / W
                y = (bbox[1]+bbox[3]/2) / H
                w = bbox[2] / W
                h = bbox[3] / H
                
                f.write(f'{category_id} {x} {y} {w} {h}\n')
                
                
print("number of train set :", n)

number of train set : 31376
